# ********** next try

In [25]:
import numpy as np
import pandas as pd
import csv
import tweepy
import os
import re

In [4]:
retweets = pd.read_csv('HealthRelease_retweets.csv')
retweet_users = retweets['retweet.user_id'].tolist()
#retweet_users

In [56]:
#Twitter API credentials
consumer_key = "z0MVIcGiFOeJ3Qdqw9J0fdFHA"
consumer_secret = "vVUz4VIsEyEWBjv44djVmcBQfaHuxvl4sYEGlgDSDlTee4mQuo"
access_key = "2230096484-TTUpaupKXLQ0LGZ51y0yiIYZ3KgZwIlHigOMf1P"
access_secret = "Ru3nGZXiyvczEkTt3ehUreeTpoESVSejjslz88NbINefJ"

In [44]:
protected_retweeters = []
count = 0
def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method
    
    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    
    #initialize a list to hold all the tweepy Tweets
    alltweets = []  
    
    try:
        #make initial request for most recent tweets (200 is the maximum allowed count)
        new_tweets = api.user_timeline(id = screen_name,count=200)
    
        #save most recent tweets
        alltweets.extend(new_tweets)
    
        #save the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1
    
        #keep grabbing tweets until there are no tweets left to grab
        while len(new_tweets) > 0:
            #print(f"getting tweets before {oldest}")

            #all subsiquent requests use the max_id param to prevent duplicates
            new_tweets = api.user_timeline(id = screen_name,count=200,max_id=oldest)

            #save most recent tweets
            alltweets.extend(new_tweets)

            #update the id of the oldest tweet less one
            oldest = alltweets[-1].id - 1

            #print(f"...{len(alltweets)} tweets downloaded so far")

        #transform the tweepy tweets into a 2D array that will populate the csv 
        outtweets = [[tweet.id_str, tweet.created_at, tweet.text] for tweet in alltweets]

        #write the csv  
        with open(f'user_past/{screen_name}_tweets.csv', 'w') as f:
            writer = csv.writer(f)
            writer.writerow(["id","created_at","text"])
            writer.writerows(outtweets)
        pass
    
    except:
        #count = count + 1
        print("Found protected users")
        protected_retweeters.append(screen_name)


if __name__ == '__main__':
    #get_all_tweets(165124122)
    for i in retweet_users:
         get_all_tweets(i)

Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users
Found protected users


In [48]:
import os
os.chdir('user_past')


In [53]:
length = [os.listdir()]
len(length[0])

1389

In [54]:
len(protected_retweeters)

20

In [45]:
# authorization of consumer key and consumer secret
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
  
# set access to user's access key and access secret 
auth.set_access_token(access_token, access_token_secret)
  
# calling the api 
api = tweepy.API(auth)
  
# the ID of the user
id = 4865126848
  
# fetching the user
user = api.get_user(id)
  
# fetching the screen name
screen_name = user.screen_name
  
print("The screen name of the user is : " + screen_name)


The screen name of the user is : NickSimpson_8


# CRED RANK

In [73]:
#retweet_users

In [69]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
protected_users = []
user_details = []
for _id in retweet_users:
    #print(_id)
    try:
        user = api.get_user(str(_id))
        user_details.append(user)
    except:
        protected_users.append(_id)

In [105]:
type(user_details)
with open("file.txt", "w") as output:
    output.write(str(user_details))

In [107]:
#protected_users

In [ ]:
user_details_df = pd.DataFrame(columns=["user_id", "screen_name", "user_description", "user_location", "retweeted_text", "hashtags"])

# SCARLET - Sentiment score & Uncertainity calculation

Sentiment calculation

In [1]:
import numpy as np
import pandas as pd
import csv
import tweepy
import os
import re
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings('ignore')

In [2]:
retweets = pd.read_csv('HealthRelease_retweets.csv')
retweet_users = retweets['retweet.user_id'].tolist()

In [3]:
retweetuser_record = pd.DataFrame()
retweetuser_record['ID'] = retweet_users
retweetuser_record['positive_percent'] = ""
retweetuser_record['negative_percent'] = ""
retweetuser_record['neutral_percent'] = ""

In [ ]:
new_index=-1
protected_users = []
other_errors = []
for x in retweet_users:
    #indexing the empty dataframe
    new_index=new_index+1
    
    #getting necessary details from retweets csv file
    retweet_userid = x
    retweet_createdat = retweets[retweets['retweet.user_id'] == retweet_userid]['retweet.created_at']
    retweeted_text = retweets[retweets['retweet.user_id'] == retweet_userid]['retweet.text']
    print("Starting ", retweet_userid)
    
    try:
        #loading the user's past tweets csv
        os.chdir('/Users/sowmiya/Desktop/user_past')
        file_path = str(retweet_userid)+'_tweets.csv'
        tweets_history = pd.read_csv(file_path)

        #tweets cleaning
        def remove_pattern(text, pattern_regex):
            r = re.findall(pattern_regex, text)
            for i in r:
                text = re.sub(i, '', text)
            return text
        tweets_history['text_cleaned'] = np.vectorize(remove_pattern)(tweets_history['text'], "@[\w]*: | *RT*")
        tweets_history['text_cleaned'] = np.vectorize(remove_pattern)(tweets_history['text_cleaned'], "@[\w]* | *RT*")
        cleaned_tweets = []
        for index, row in tweets_history.iterrows():
            words_without_links = [word for word in row.text_cleaned.split() if 'http' not in word]
            cleaned_tweets.append(' '.join(words_without_links))
        tweets_history['text_cleaned'] = cleaned_tweets
        tweets_history['text_cleaned'] = tweets_history['text_cleaned'].str.replace("[^a-zA-Z# ]", "")

        #sentiment calculation
        def sentiment_scores(sentence):
            sid_obj = SentimentIntensityAnalyzer()
            sentiment_dict = sid_obj.polarity_scores(sentence)
            return sentiment_dict['compound']

        tweets_history['sentiment_score'] = ''
        positive = 0
        negative = 0
        neutral = 0

        for i in range(len(tweets_history)):
            try:
                tweets_history['sentiment_score'][i] = sentiment_scores(tweets_history['text_cleaned'][i])
                if tweets_history['sentiment_score'][i] >= 0.05 :
                    positive = positive+1

                elif tweets_history['sentiment_score'][i] <= - 0.05 :
                    negative = negative+1
                else :
                    neutral = neutral+1
            except:
                tweets_history['sentiment_score'][i] = 0

        positive = positive/len(tweets_history)
        positive = round((positive*100), 2)

        negative = negative/len(tweets_history)
        negative = round((negative*100), 2)

        neutral = neutral/len(tweets_history)
        neutral = round((neutral*100), 2)

        #save the records
        retweetuser_record['positive_percent'][new_index] = positive
        retweetuser_record['negative_percent'][new_index] = negative
        retweetuser_record['neutral_percent'][new_index] = neutral
        tweets_history.to_csv(str(retweet_userid)+'_tweets.csv')


        print(positive, negative, neutral)
        print("Done with ", retweet_userid)
        print("---------------------------\n")
        
    except FileNotFoundError:
        print("File Not Found\n--------------------------\n")
        protected_users.append(x)
        
    except:
        print("Other error\n--------------------------\n")
        other_errors.append(x)
        
        

Starting  4865126848
49.26 12.21 38.53
Done with  4865126848
---------------------------

Starting  868150893140684800
6.93 18.67 74.4
Done with  868150893140684800
---------------------------

Starting  875351948
Other error
--------------------------

Starting  165124122
File Not Found
--------------------------

Starting  2345443888
61.12 15.47 23.41
Done with  2345443888
---------------------------

Starting  243953821
22.16 12.98 64.86
Done with  243953821
---------------------------

Starting  740243460880490496
Other error
--------------------------

Starting  745740125112512512
6.27 24.75 68.98
Done with  745740125112512512
---------------------------

Starting  761217071762006016
45.42 12.3 42.28
Done with  761217071762006016
---------------------------

Starting  834499897521012736
65.61 6.14 28.25
Done with  834499897521012736
---------------------------

Starting  1608652920
43.6 10.14 46.26
Done with  1608652920
---------------------------

Starting  414459679
48.09 15.62 

45.33 20.63 34.03
Done with  28142720
---------------------------

Starting  19898016
47.64 12.0 40.36
Done with  19898016
---------------------------

Starting  480586426
28.76 38.73 32.52
Done with  480586426
---------------------------

Starting  257095509
33.95 6.84 59.21
Done with  257095509
---------------------------

Starting  1671405726
46.98 17.74 35.28
Done with  1671405726
---------------------------

Starting  620561527
54.48 21.15 24.37
Done with  620561527
---------------------------

Starting  4766057477
35.28 34.11 30.61
Done with  4766057477
---------------------------

Starting  16275858
53.68 22.93 23.39
Done with  16275858
---------------------------

Starting  20827316
52.74 18.69 28.57
Done with  20827316
---------------------------

Starting  49790780
31.97 30.92 37.11
Done with  49790780
---------------------------

Starting  1407952194
2.16 1.47 96.37
Done with  1407952194
---------------------------

Starting  46825188
46.54 21.05 32.41
Done with  46825188
--

38.87 18.99 42.14
Done with  22626973
---------------------------

Starting  586751692
33.36 37.31 29.32
Done with  586751692
---------------------------

Starting  49996380
42.78 23.07 34.15
Done with  49996380
---------------------------

Starting  2706724184
1.61 0.96 97.43
Done with  2706724184
---------------------------

Starting  505218318
31.18 27.32 41.5
Done with  505218318
---------------------------

Starting  1120710528
29.09 40.66 30.26
Done with  1120710528
---------------------------

Starting  205804929
58.53 11.32 30.15
Done with  205804929
---------------------------

Starting  305934961
29.4 22.96 47.64
Done with  305934961
---------------------------

Starting  50717212
43.51 27.83 28.66
Done with  50717212
---------------------------

Starting  150934954
42.86 18.88 38.27
Done with  150934954
---------------------------

Starting  2683898904
47.52 23.05 29.42
Done with  2683898904
---------------------------

Starting  70292285
45.02 19.7 35.28
Done with  70292285

51.62 16.03 32.35
Done with  804151878309310465
---------------------------

Starting  574878119
46.16 13.57 40.27
Done with  574878119
---------------------------

Starting  714591936925999104
34.68 33.85 31.47
Done with  714591936925999104
---------------------------

Starting  103227486
12.79 6.11 81.09
Done with  103227486
---------------------------

Starting  637219415
6.83 31.29 61.88
Done with  637219415
---------------------------

Starting  701128568957370368
52.65 14.38 32.97
Done with  701128568957370368
---------------------------

Starting  422724605
43.72 15.88 40.4
Done with  422724605
---------------------------

Starting  17602018
53.34 14.81 31.85
Done with  17602018
---------------------------

Starting  2834393764
56.58 9.22 34.2
Done with  2834393764
---------------------------

Starting  1155192127
40.37 25.22 34.4
Done with  1155192127
---------------------------

Starting  53957448
3.31 1.59 95.09
Done with  53957448
---------------------------

Starting  13827

61.5 10.21 28.29
Done with  619214515
---------------------------

Starting  918570705540845570
29.31 3.51 67.18
Done with  918570705540845570
---------------------------

Starting  4833240020
47.74 21.62 30.64
Done with  4833240020
---------------------------

Starting  429028516
45.14 23.79 31.07
Done with  429028516
---------------------------

Starting  969905531866533890
36.61 51.81 11.58
Done with  969905531866533890
---------------------------

Starting  771511754
38.39 19.56 42.05
Done with  771511754
---------------------------

Starting  754794407035936773
35.22 18.44 46.34
Done with  754794407035936773
---------------------------

Starting  328724776
8.49 23.6 67.91
Done with  328724776
---------------------------

Starting  962765306128089088
62.0 5.24 32.76
Done with  962765306128089088
---------------------------

Starting  54179692
41.78 31.46 26.76
Done with  54179692
---------------------------

Starting  111412750
27.6 30.61 41.79
Done with  111412750
----------------

...

68.99 10.72 20.29
Done with  1472482507
---------------------------

Starting  360662772
74.67 7.94 17.39
Done with  360662772
---------------------------

Starting  884096843998609412
43.46 13.75 42.78
Done with  884096843998609412
---------------------------

Starting  243593548
0.47 0.12 99.41
Done with  243593548
---------------------------

Starting  1319195282
28.22 23.87 47.92
Done with  1319195282
---------------------------

Starting  4425663027
38.03 18.53 43.44
Done with  4425663027
---------------------------

Starting  470099461
23.07 18.23 58.7
Done with  470099461
---------------------------

Starting  1166611279
59.1 12.74 28.16
Done with  1166611279
---------------------------

Starting  4425663027
38.03 18.53 43.44
Done with  4425663027
---------------------------

Starting  2982739121
48.02 15.82 36.16
Done with  2982739121
---------------------------

Starting  4425663027
38.03 18.53 43.44
Done with  4425663027
---------------------------

Starting  1031178019312988

34.91 32.1 32.99
Done with  107173519
---------------------------

Starting  821964998
40.33 25.79 33.88
Done with  821964998
---------------------------

Starting  2339138737
21.29 22.24 56.47
Done with  2339138737
---------------------------

Starting  84680854
44.93 23.56 31.51
Done with  84680854
---------------------------

Starting  2463854064
30.85 37.03 32.12
Done with  2463854064
---------------------------

Starting  15652824
31.72 36.44 31.84
Done with  15652824
---------------------------

Starting  797456892452859904
39.88 33.87 26.26
Done with  797456892452859904
---------------------------

Starting  204423673
36.7 38.49 24.82
Done with  204423673
---------------------------

Starting  232217386
40.44 27.18 32.39
Done with  232217386
---------------------------

Starting  44160485
36.76 32.29 30.95
Done with  44160485
---------------------------

Starting  3105174602
35.09 33.21 31.7
Done with  3105174602
---------------------------

Starting  327697369
38.98 40.18 20.83

53.62 15.39 30.99
Done with  408713491
---------------------------

Starting  619483557
55.72 9.5 34.78
Done with  619483557
---------------------------

Starting  789254
22.35 21.73 55.92
Done with  789254
---------------------------

Starting  36312784
46.7 16.33 36.97
Done with  36312784
---------------------------

Starting  217167157
37.42 18.32 44.26
Done with  217167157
---------------------------

Starting  165387035
53.24 22.47 24.3
Done with  165387035
---------------------------

Starting  721551261925908481
File Not Found
--------------------------

Starting  264191286
44.78 23.32 31.9
Done with  264191286
---------------------------

Starting  4834628099
22.54 3.76 73.7
Done with  4834628099
---------------------------

Starting  534175893
45.71 16.38 37.91
Done with  534175893
---------------------------

Starting  727165278170288128
54.18 8.4 37.42
Done with  727165278170288128
---------------------------

Starting  2699418795
57.84 15.07 27.09
Done with  2699418795
----

32.62 34.23 33.15
Done with  345039699
---------------------------

Starting  54446565
30.51 17.13 52.36
Done with  54446565
---------------------------

Starting  769254015582470144
21.04 12.3 66.66
Done with  769254015582470144
---------------------------

Starting  2202946701
15.98 25.33 58.69
Done with  2202946701
---------------------------

Starting  826141378635776004
42.16 20.66 37.18
Done with  826141378635776004
---------------------------

Starting  558185944
39.87 23.0 37.13
Done with  558185944
---------------------------

Starting  272603881
38.4 23.43 38.17
Done with  272603881
---------------------------

Starting  4273129752
56.4 15.26 28.34
Done with  4273129752
---------------------------

Starting  65762950
14.47 17.72 67.81
Done with  65762950
---------------------------

Starting  3138821804
57.81 4.9 37.29
Done with  3138821804
---------------------------

Starting  4452439762
23.74 4.35 71.91
Done with  4452439762
---------------------------

Starting  819356300

36.28 20.43 43.29
Done with  969197651534270464
---------------------------

Starting  880228342951292930
62.5 6.25 31.25
Done with  880228342951292930
---------------------------

Starting  785470495550500865
51.53 18.28 30.19
Done with  785470495550500865
---------------------------

Starting  915353415752044544
26.04 12.1 61.86
Done with  915353415752044544
---------------------------

Starting  525201158
40.08 13.28 46.64
Done with  525201158
---------------------------

Starting  2220065664
67.34 9.5 23.16
Done with  2220065664
---------------------------

Starting  89203052
25.15 23.92 50.93
Done with  89203052
---------------------------

Starting  892858064075968513
25.01 38.39 36.6
Done with  892858064075968513
---------------------------

Starting  2582379270
46.49 5.41 48.1
Done with  2582379270
---------------------------

Starting  903801997
35.58 33.18 31.24
Done with  903801997
---------------------------

Starting  954087554537541632
42.13 24.31 33.56
Done with  9540875

In [ ]:
retweetuser_record

In [ ]:
protected_users

In [ ]:
other_errors